Refactoring lost histogram processing file - IT IS RECOVERED!!!

load all packages
load up IA file as dataframe
load up joined shapefile with census info
loop trhough each row, write to a new file, new headers, as well as old, for each new variable create it by saying if a value of a row's column so fow[23] for example is great than or equal to xyz then the new variable is true or false respectively, add new variable value to row, then write all to new csv file
drop unneeded columns
dg loc so its only 50 rows
change price to be a float and without funny symbols
create new variables (that you'll find listed out on the csv file from yesterday)

when you have the new variable in the new csv file, lets load that into qgis and join it again to the census shapefile, watch filenames
then we'll have a new variables, with census data, geolocated file! all in one place! 
then drop more columns that don't need (from census)

In [1]:
import geojson
import geopandas
import pandas as pd
import numpy as np
import statistics as st
import matplotlib as plt
import scipy
from scipy import stats
from scipy.stats import describe
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import seaborn as sns
import folium
import csv
from IPython.display import display, HTML
from pathlib import Path 
import os
from datetime import date, time, datetime
from dateutil import tz #for dealing with timezone
from geopy import distance

/Users/stateofplace/.virtualenvs/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [30]:
def get_time():
    today = date.today()
    current_time = time(datetime.now().hour, datetime.now().minute, datetime.now().second)
    date_today = datetime.combine(today, current_time)
    new_date_time = str(date_today).replace(" ", "_").replace("-", "_").replace(":", "_")
    return(new_date_time)
print(get_time())
new_date_time = get_time()

2022_03_24_18_57_59


In [38]:
def load_files(ia, ia_old, census_ia): 
    #load and create dataframes
    ia_df = pd.read_csv(ia)
    ia_df_old = pd.read_csv(ia_old)
    #drop
    ia_df.drop(columns=["scrape_id", "bathrooms_text","host_verifications", "host_neighbourhood","property_type", "amenities", "host_verifications", "neighbourhood", "host_listings_count", "picture_url","host_url", "description", "neighborhood_overview", "host_about", "host_response_time", "host_response_rate", 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_has_profile_pic', 'host_identity_verified', 'calendar_updated', 'has_availability', 'calendar_last_scraped', 'number_of_reviews', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'neighbourhood_group_cleansed', 'beds', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'bathrooms', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], inplace=True)
    #rename columns ? 
    #trim columns 
    #ia_df = ia_df.loc[:50, :]
    ia_df_old.drop(columns=["scrape_id", "bathrooms_text","host_verifications", "host_neighbourhood","property_type", "amenities", "host_verifications", "neighbourhood", "host_listings_count", "picture_url","host_url", "description", "neighborhood_overview", "host_about", "host_response_time", "host_response_rate", 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_has_profile_pic', 'host_identity_verified', 'calendar_updated', 'has_availability', 'calendar_last_scraped', 'number_of_reviews', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'neighbourhood_group_cleansed', 'beds', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'bathrooms', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], inplace=True)

    #handle shape file joined, we need it for some variable creation, but doens't need to be a airbnb joined one, just the census info is fine
    census_gdf = geopandas.read_file(census_ia)
    return(ia_df, ia_df_old, census_gdf)
#ia_df_clean_old, census_gdf = load_files("Out_CSV/listings_9_1_2020_prepand.csv", "../thesis_app/airbnb_census_joined.shp")
ia_df_clean, ia_df_clean_old, census_gdf = load_files("../scrap_work/florence_listings_jan_27.csv", "Out_CSV/listings_9_1_2020_prepand.csv", "../thesis_app/airbnb_census_joined.shp") #using the shapefile with no airbnb data yet makes more sense

In [39]:
#will only work once i think
def fixpricing_file(dataframe):
    #price adjust
    dataframe['price'] = dataframe['price'].str.replace("$", "").str.replace(',', '')
    dataframe['price'] = pd.to_numeric(dataframe['price'])
    #tests
    # print(dataframe['price']) #no special characters 
    # print(type(dataframe['price'])) #float not string 
    return dataframe

ia_df_clean = fixpricing_file(ia_df_clean) #ia_df_clean 


/var/folders/9h/m6366gw17qd8jtbtczp7ttnc0000gp/T/ipykernel_6053/1573565530.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe['price'] = dataframe['price'].str.replace("$", "").str.replace(',', '')


In [40]:
def df_to_csv(dataframe):
    filepath = Path('../thesis_app/Out_CSV/ia_1_27_22' + new_date_time + '.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True) 
    dataframe.to_csv(filepath)
    #tests
    print(filepath)
    return csv
df_to_csv(ia_df_clean)   #ia_df_clean ia_df_clean_old

../thesis_app/Out_CSV/ia_1_27_222022_03_24_18_57_59.csv


<module 'csv' from '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/csv.py'>

In [36]:
# ia_df_clean_old.head()
# #s.loc[s['has_liscense'] == 0]
# row_old_df = ia_df_clean_old.loc[ia_df_clean_old['id'] == 24469]

# row_old_df['host_id']
ia_df_clean.head()


,id,listing_url,last_scraped,name,host_id,host_name,host_since,host_location,host_total_listings_count,neighbourhood_cleansed,...,availability_30,availability_60,availability_90,availability_365,number_of_reviews_ltm,number_of_reviews_l30d,license,instant_bookable,calculated_host_listings_count,reviews_per_month
0,24469,https://www.airbnb.com/rooms/24469,2021-12-17,Fortezza/City Centre Modern Apt 2+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,26,56,86,361,0,0,NaN,f,4,0.04
1,24470,https://www.airbnb.com/rooms/24470,2021-12-17,Fortezza/City Centre Modern Apt 2+1,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,0,0,0,107,0,0,NaN,f,4,0.02
2,24471,https://www.airbnb.com/rooms/24471,2021-12-17,Fortezza/City Centre Modern Apt 4+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,0,0,0,107,0,0,NaN,f,4,NaN
3,24472,https://www.airbnb.com/rooms/24472,2021-12-17,Fortezza/City Centre Modern Apt 4+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,Centro Storico,...,26,56,86,361,0,0,NaN,f,4,0.02
4,26738,https://www.airbnb.com/rooms/26738,2021-12-17,N4U Guest House Florence,113883,N4U Guest House,2010-04-26,"Florence, Toscana, Italy",2,Centro Storico,...,12,42,72,347,0,0,NaN,f,2,0.22


In [48]:

def create_new_variables(cleaned_file_input, file_output, dataframe, dataframe_old):
    #do a lot of processing
    all = []
    with open(cleaned_file_input, 'r') as file:
        with open(file_output, 'w') as newfile:
            csvwriter = csv.writer(newfile,lineterminator='\n')
            csvreader = csv.reader(file)
            #extract headers
            header = next(csvreader)
            new_headers = ['days_rented', 'rounded_revenue', 'is_hotel', 'is_entire', 'many_listings', 'only_1_listing', 'only_2_listings', 'host_florence', 'has_liscense', 'is_instant_bookable', 'global_total_listings', 'dist_duomo', 'is_centro', 'is_gavinana', 'is_isolotto', 'is_rifredi', 'is_campo', 'listing_revenue_exceed_LTR', 'effected_by_policy_1', 'effected_by_policy_2', 'effected_by_policy_3', 'effected_by_policy_4', 'commercial']
            for item in new_headers:
                header.append(item)
            all.append(header)
            for row in csvreader:
                #index row 
                listing_row_id = int(row[0])
                listing_id = int(row[1])
                # total global and local listings
                global_total_listings = int(row[9]) + int(row[25]) 
                #dist_duomo
                duomo_coords = (43.7731, 11.2560)
                #listing_coords = lat, long
                listing_coords = (row[11], row[12])
                ##dist_duomo = duomo_coords - listing_coords
                dist_duomo = (distance.distance(duomo_coords, listing_coords).m)
                #has_liscense
                if len(row[23]) > 0:
                    has_liscense = 1
                else:
                    has_liscense = 0
                #is_instant_bookable
                if row[24] == 't':
                    is_instant_bookable = 1
                else:
                    is_instant_bookable = 0
                #getting info for "host_florence" success
                if ("Florence" or "Firenze") in row[8]:
                    host_florence = 1
                else:
                    host_florence = 0
                    #print(host_florence)
                #getting info for "is_centro" success
                #get better average rent data just guessed from idealista
                if "Centro" in row[10]:
                    is_centro = 1
                    neighborhood_rent = 1000 * 12
                else:
                    is_centro = 0
                #is_campo success
                if "Campo" in row[10]:
                    is_campo = 1
                    neighborhood_rent = 700 * 12
                else:
                    is_campo = 0
                #is_rifredi success
                if "Rifredi" in row[10]:
                    is_rifredi = 1
                    neighborhood_rent = 600 * 12
                else:
                    is_rifredi = 0
                if "Isolotto" in row[10]:
                    is_isolotto = 1
                    neighborhood_rent = 400 * 12
                else:
                    is_isolotto = 0
                if "Gavinana" in row[10]:
                    is_gavinana = 1
                    neighborhood_rent = 900 * 12
                else:
                    is_gavinana = 0
                #many_listings variable success
                if int(row[9]) > 2:
                    many_listings = 1
                else:
                    many_listings = 0
                #only_1 success
                if int(row[9]) == 1:
                    only_1_listing = 1
                else: 
                    only_1_listing = 0
                #only_2 success
                if int(row[9]) == 2:
                    only_2_listings = 1
                else:
                    only_2_listings = 0
                #is_entire success 
                if row[13] == 'Entire home/apt':
                    is_entire = 1
                else:
                    is_entire = 0
                #is_hotel success
                if row[13] == 'Hotel room':
                    is_hotel = 1
                else:
                    is_hotel = 0
                #days_rented
                #year and month
                date_scraped = row[3]
                #tests
                #print(type(date_scraped))
                #print(date_scraped.split('-'))
                year = date_scraped.split('-')[0]
                month = date_scraped.split('-')[1]
                day = date_scraped.split('-')[2]
                #year started 
                year_started = int(row[7].split('-')[0])
                month_started = int(row[7].split('-')[1])
                #TODO check against boston and CA gov method, UPDATE BASED ON MURRAY SUGGESTION
                #did reviews a year divided by how likely someone gives a review (airbnb data) * avg stay in florence (2.6)
                #fix days rented so can use in policy 4 and base it on murray's suggestion
                # days_rented = (int(row[21])/.7)*2.6
                # if old_listing:
                if (year_started < 2019):
                    #couldve been more exact like 2020 and may but too complicated ... 
                    #old listing
                    # pass
                    # row_old_df = ia_df_clean_old.loc[ia_df_clean_old['id'] == 24469]
                    # row_old_df['host_id']
                    old_df_ltm_row = dataframe_old.loc[dataframe_old['id'] == listing_id]
                    #listing_id, 'number_of_reviews_ltm']
                    old_df_ltm = old_df_ltm_row['number_of_reviews_ltm']
                    
                    old_df_ltm = pd.to_numeric(old_df_ltm_row['number_of_reviews_ltm'])
                    # print(old_df_ltm)
                    # print(type(old_df_ltm))
                    days_rented_prepan = ((old_df_ltm)/.5)*3.2 #load csv file here from 9/1/2020 because if all are started at least in 2018 then they'd be in there in 2020 pre pand, bit of a buffer... also will use dataframe and look up using .loc with row being the listing id of this and column being the review
                    days_rented_postpan = (int(row[21])/.5)*3.2
                    days_rented = (days_rented_prepan + days_rented_postpan)/2
                    days_rented = pd.to_numeric(days_rented)
                #     #do avg of pre covid post covid data based on yearly review
                # if new_listing:
                elif year_started >= 2019:
                    #new listing 
                    # pass
                    # #use monthly review
                    days_rented = (int(row[22])/.5)*3.2
                
                #for now... making it more conservative with the venice lady's .5, and changing to 3.2 based on statistica 
                #days_rented = (int(row[21])/.5)*3.2
                #thought I handeled this in the price fix def
                #revenue #have to strip the string for price then turn into float
                price_stripped_dollar = row[16].strip('$')
                price_replaced_comma = price_stripped_dollar.replace(',','')
                new_price = float(price_replaced_comma)
                #yearly revenue
                revenue = days_rented * new_price
                rounded_revenue = round(revenue, 2)

                #commercial
                if dataframe.loc[listing_row_id, 'host_total_listings_count'] > 2:
                    commercial = 1
                elif host_florence == 0:
                    commercial = 1
                else:
                    commercial = 0
                #very_likely_commercial
                if dataframe.loc[listing_row_id, 'host_total_listings_count'] > 2 and dataframe.loc[listing_row_id, 'instant_bookable'] == 1:
                    very_likely_commercial = 1
                else:
                    very_likely_commercial = 0

                #effected_by_policy_x (1-liscense,2- -is entire, 3-2 listing max, 4-day limit 90) #switched 2 and 4 from origingal 
                if license == 0:
                    effected_by_policy_1 = 1
                else: 
                    effected_by_policy_1 = 0
                if is_entire == 1:
                    effected_by_policy_2 = 1
                else:
                    effected_by_policy_2 = 0
                #man_listings is more than 2 
                if many_listings == 1:
                    effected_by_policy_3 = 1
                else:
                    effected_by_policy_3 = 0
                ##TODO truth value of series unclear... issue not going to fix righ tnow.. 
                # if days_rented > 90:
                #     effected_by_policy_4 = 1
                # else:
                #     effected_by_policy_4 = 0
                effected_by_policy_4 = 'unsure'
                #listing_revenue_exceed_LTR
                #if yearly revenue exceeds average rent per neighborhood x12
                #have to skip first one where its '' TODO: make sure for sure its only the first row that is throwing htis error but for now we don't even use these variables so not worried 
                #TODO issue here all of a sudden with truth value error connected to update on days rented maybe? 
                # try: yearly_revenue_room = rounded_revenue / float(row[15])
                # # print('yearly_revenue_room is:' + str(yearly_revenue_room))
                # except Exception:
                #     pass
                # neighborhood_LTR_room = neighborhood_rent
                # # print('neighborhood_LTR_room is:' + str(neighborhood_LTR_room))
                # if yearly_revenue_room > neighborhood_LTR_room:
                #     listing_revenue_exceed_LTR = 1
                # else:
                #     listing_revenue_exceed_LTR = 0
                # print('is listing_revenue_exceed_LTR true?:' + str(listing_revenue_exceed_LTR))
                listing_revenue_exceed_LTR = 'unsure'
                new_variables = [days_rented, rounded_revenue, is_hotel, is_entire, many_listings, only_1_listing, only_2_listings, host_florence, has_liscense, is_instant_bookable, global_total_listings, dist_duomo, is_centro, is_gavinana, is_isolotto, is_rifredi, is_campo, listing_revenue_exceed_LTR, effected_by_policy_1, effected_by_policy_2, effected_by_policy_3, effected_by_policy_4, commercial]
                for item in new_variables:
                    row.append(item)
                #adding entire row to all 
                all.append(row)
            #writing all rows to new out file
            csvwriter.writerows(all)
    return csvwriter
create_new_variables('../thesis_app/Out_CSV/ia_1_27_222022_03_23_18_40_54.csv', '../thesis_app/Out_CSV/ia_1_27_222022_03_23_18_40_54_new_variables.csv', ia_df_clean, ia_df_clean_old)

In [28]:
def create_geocensus_variables(ia_newvar_file_input, file_output, geodataframe):
#need the joined gdf with airbnb data for the stats on census because we use the listing row id of the airbnb file which only matches with the gdf because it has airnbb data in it... maybe not necessary these variables at all anyway...we loop through csv file with 51 airbnb lisitings...to match with census info.. 
    all = []
    with open(ia_newvar_file_input, 'r') as file:
        with open(file_output, 'w') as newfile:
            csvwriter = csv.writer(newfile,lineterminator='\n')
            csvreader = csv.reader(file)
            #extract headers
            header = next(csvreader)
            new_headers = ['census_tract_id', 'num_residents_census', 'num_households_census', 'res_buildings_census', 'num_bedrooms_census', 'num_household_renters_census', 'num_household_owners_census']
            for item in new_headers:
                header.append(item)
            all.append(header)
            for row in csvreader:
            #index row 
                listing_row_id = int(row[0])
            
                #census_tract_id_sez2011
                census_tract_id = geodataframe.loc[listing_row_id, 'SEZ2011']
                #census data points 
                columns = ['P1', 'A47', 'A46', 'PF1', 'E3', 'E27', 'SEZ2011']
                #number of residents per census tract P1
                num_residents_census = (geodataframe.loc[listing_row_id, 'P1'])
                #number of households per census tract PF1
                num_households_census = (geodataframe.loc[listing_row_id, 'PF1'])
                #number of residential buildings per census tract E3
                res_buildings_census = (geodataframe.loc[listing_row_id, 'E3'])
                # number of bedrooms per census tract E27
                num_bedrooms_census = (geodataframe.loc[listing_row_id, 'E27'])
                #number of household renters per census tract  A46 
                num_household_renters_census = (geodataframe.loc[listing_row_id, 'A46'])
                #number of household homeowners per census tract A47
                num_household_owners_census = (geodataframe.loc[listing_row_id, 'A47'])
                #adding new variable to row for each row in file
                
                new_variables = [census_tract_id, num_residents_census, num_households_census, res_buildings_census, num_bedrooms_census, num_household_renters_census, num_household_owners_census]
                for item in new_variables:
                    row.append(item)
                #adding entire row to all 
                all.append(row)
            #writing all rows to new out file
            csvwriter.writerows(all)
    return csvwriter
create_geocensus_variables('../thesis_app/Out_CSV/ia_1_27_222022_03_23_18_40_54_new_variables.csv', '../thesis_app/Out_CSV/ia_1_27_222022_03_23_18_40_54_new_variables_census.csv', census_gdf)